In [1]:
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
import pandas as pd
import mne
data_dir = 'data_meg'
subj = "R2490"
dataqual = 'prepro' #or loc/exp
exp = 'exp' #or exp
dtype = "raw"
label_dir = 'data_log'
save_dir = 'data_meg'



In [2]:
raw = mne.io.read_raw_fif('data_meg/R2490/prepro/R2490_exp.fif', preload='temp_raw.fif')

Opening raw data file data_meg/R2490/prepro/R2490_exp.fif...


/tmp/ipykernel_3327471/636745299.py:1: RuntimeWarning: This filename (data_meg/R2490/prepro/R2490_exp.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif('data_meg/R2490/prepro/R2490_exp.fif', preload='temp_raw.fif')


    Range : 0 ... 2779999 =      0.000 ...  2779.999 secs
Ready.
Opening raw data file /home/my2689/.ssh/meg_decoding/data_meg/R2490/prepro/R2490_exp-1.fif...
    Range : 2780000 ... 3669999 =   2780.000 ...  3669.999 secs
Ready.
Reading 0 ... 3669999  =      0.000 ...  3669.999 secs...


In [3]:
sfreq = raw.info['sfreq']
raw.filter(1, 40, method='iir')

downsample = 10

Filtering raw data in 2 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 1.00, 40.00 Hz: -6.02, -6.02 dB



In [4]:
events = mne.find_events(raw, stim_channel='STI 014', output='onset', shortest_event=1)
event_id = {
    'start': 160,
    'move': 161,
    'reveal_red': 162,
    'reveal_white': 163,
    'done': 164,
    'choice': 165,
    'timeout': 166
}

# Define trials to remove
trials_to_remove = []

4032 events found on stim channel STI 014
Event IDs: [160 161 162 163 164 166]


In [5]:
start_events = events[events[:, 2] == event_id['start']]
done_events = events[events[:, 2] == event_id['done']]
timeout_events = events[events[:, 2] == event_id['timeout']]
reveal_red_events = events[events[:, 2] == event_id['reveal_red']]
reveal_white_events = events[events[:, 2] == event_id['reveal_white']]
sfreq = raw.info['sfreq']  # Sampling frequency

# Combine 'done' and 'timeout' events
end_events = np.concatenate((done_events, timeout_events))
end_events = end_events[end_events[:, 0].argsort()]  # Sort by time

# Initialize a list to store trial information
trial_info = []
previous_start_sample = None
processed_starts = set()
start_idx = 0

# Iterate through each start event to create trial information
for idx, start_event in enumerate(start_events):
    start_sample = start_event[0]

    # Find the next end event after the start event
    end_idx = np.searchsorted(end_events[:, 0], start_sample, side='right')
    if end_idx < len(end_events):
        end_sample = end_events[end_idx, 0]

        # Check if the current start is at least 20 seconds after the previous start
        if (previous_start_sample is None or (start_sample - previous_start_sample) / sfreq >= 20) and start_sample not in processed_starts:
            # Proceed with processing this start event
            previous_start_sample = start_sample
            processed_starts.add(start_sample)  # Add to the set of processed starts

            # Calculate tmin and tmax for the epoch
            tmin = -0.2  # 0.2 s before 'start'
            tmax = (end_sample - start_sample) / sfreq + 1.0  # Duration from 'start' to 1 s after end event
            print(f"tmax is {tmax}")

            reveal_red_within_trial = reveal_red_events[(reveal_red_events[:, 0] > start_sample) & 
                                                        (reveal_red_events[:, 0] < end_sample)]
            reveal_white_within_trial = reveal_white_events[(reveal_white_events[:, 0] > start_sample) & 
                                                            (reveal_white_events[:, 0] < end_sample)]

            # Store trial information, including whether 'reveal_red' and 'reveal_white' occurred
            trial_info.append({
                'event_sample': start_sample,
                'trial_index': start_idx,
                'duration': tmax,
                'tmin': tmin,
                'tmax': tmax,
                'done': len(done_events) > 0,
                'start_times': start_sample / sfreq,
                'reveal_red': len(reveal_red_within_trial) > 0,  # Boolean flag indicating if 'reveal_red' occurred
                'reveal_red_times': (reveal_red_within_trial[:, 0] - start_sample) / sfreq if len(reveal_red_within_trial) > 0 else [],
                'reveal_white': len(reveal_white_within_trial) > 0,  # Boolean flag indicating if 'reveal_white' occurred
                'reveal_white_times': (reveal_white_within_trial[:, 0] - start_sample) / sfreq if len(reveal_white_within_trial) > 0 else [],
                'reveal_times': sorted(
                    ((reveal_red_within_trial[:, 0] - start_sample) / sfreq).tolist() +
                    ((reveal_white_within_trial[:, 0] - start_sample) / sfreq).tolist()
                )
            })
            start_idx += 1

tmax is 10.37
tmax is 20.844
tmax is 18.367
tmax is 30.148
tmax is 30.108
tmax is 17.427
tmax is 26.046
tmax is 16.14
tmax is 30.086
tmax is 5.747
tmax is 30.131
tmax is 18.861
tmax is 30.1
tmax is 22.326
tmax is 6.579
tmax is 30.131
tmax is 9.428
tmax is 14.223
tmax is 16.57
tmax is 30.098
tmax is 15.21
tmax is 30.097
tmax is 6.829
tmax is 6.479
tmax is 30.126
tmax is 6.632
tmax is 30.104
tmax is 5.687
tmax is 13.794
tmax is 15.89
tmax is 6.895
tmax is 30.108
tmax is 22.915
tmax is 21.546
tmax is 18.657
tmax is 8.503
tmax is 10.664
tmax is 8.812000000000001
tmax is 11.69
tmax is 30.128
tmax is 12.511
tmax is 9.211
tmax is 10.74
tmax is 30.154
tmax is 23.313
tmax is 6.251
tmax is 21.454
tmax is 15.693
tmax is 8.02
tmax is 30.13
tmax is 16.454
tmax is 13.888
tmax is 23.994
tmax is 13.911
tmax is 10.327
tmax is 30.133
tmax is 15.794
tmax is 14.118
tmax is 17.522
tmax is 10.095
tmax is 30.111
tmax is 15.147
tmax is 17.24
tmax is 6.929
tmax is 30.087
tmax is 30.116
tmax is 6.065
tmax is 30

In [7]:
new_events = np.array([[info['event_sample'], 0, event_id['start']] for info in trial_info])
tmin = -0.2  
# Initialize lists to store individual epochs data and trial information
epochs_data_list = []
trial_info_valid = []

In [8]:
# Iterate over new_events and create epochs, skipping the unwanted trials
for idx, event in enumerate(new_events):
    start_sample = event[0]
    event_id_code = event[2]
    event_time = start_sample / sfreq
    total_duration = raw.times[-1]
    print(f"Tmax is {tmax}")
    # Create the epoch
    epochs = mne.Epochs(
        raw, [event], event_id={f'event_{event_id_code}': event_id_code},
        tmin=tmin, tmax=26, preload=True,
        reject_by_annotation=False, reject=None, verbose=True
    )

    # Append valid epochs to the list
    if len(epochs) > 0:
        epochs_data_list.append(epochs.get_data()[:, :, ::downsample])
        trial_info_valid.append(trial_info[idx])
    else:
        print(f"Epoch {idx} was dropped.")
        print(f"Drop log for Epoch {idx}: {epochs.drop_log}")


Tmax is 30.103
Not setting metadata
1 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 30304 original time points ...
0 bad epochs dropped
Tmax is 30.103
Not setting metadata
1 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 30304 original time points ...
0 bad epochs dropped
Tmax is 30.103
Not setting metadata
1 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 1 events and 30304 original time points ...
0 bad epochs dropped
Tmax is 30.103
Not setting metadata
1 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using d

: 

: 

: 

In [10]:
len(epochs_data_list)

31

## Decoding

In [17]:
from mne.decoding import SlidingEstimator, cross_val_multiscore, Vectorizer
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from matplotlib import pyplot as plt
label_encoder = LabelEncoder()
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score


In [18]:
labels_df = pd.read_csv(f'{label_dir}/{subj}/label.csv')

# Step 1: Get the valid trial indices
valid_trial_indices = {info['trial_index'] for info in trial_info_valid}

# Step 2: Filter labels_df to only include valid trial indices
labels_df_filtered = labels_df[labels_df['trial_index'].isin(valid_trial_indices)]
label_dict = dict(zip(labels_df_filtered['trial_index'], labels_df_filtered['trial.rule']))

# Step 3: Extract labels for the valid trials in trial_info_valid
y_labels = []
for info in trial_info_valid:
    idx = info['trial_index']
    if idx in label_dict:
        y_labels.append(label_dict[idx])

timeout_true_indices = labels_df[labels_df['timeout'] == True]['trial_index'].values
timeout_false_indices = labels_df[labels_df['timeout'] == False]['trial_index'].values


# Convert labels to integers using label encoder
y = label_encoder.fit_transform(y_labels)

# Print the number of labels and valid trials after matching
n_labels = len(y_labels)
n_trials = len(trial_info_valid)
print(f"Number of labels after matching: {n_labels}")
print(f"Number of valid trials after matching: {n_trials}")


Number of labels after matching: 31
Number of valid trials after matching: 31


# Cross-validation

In [12]:
X = np.array([md.data for md in epochs_data_list])  # Shape: (n_epochs, n_channels, n_times)
X = X.squeeze(axis=1)


In [ ]:
n_time_points = X.shape[2]
n_classes = len(np.unique(y))
# define the Temporal generalization object
time_gen = GeneralizingEstimator(clf, n_jobs=None, scoring="roc_auc", verbose=True)

# again, cv=3 just for speed
scores = cross_val_multiscore(time_gen, X, y, cv=3, n_jobs=None)

# Mean scores across cross-validation splits
scores = np.mean(scores, axis=0)

# Plot the diagonal (it's exactly the same as the time-by-time decoding above)
fig, ax = plt.subplots()
ax.plot(epochs.times, np.diag(scores), label="score")
ax.axhline(0.5, color="k", linestyle="--", label="chance")
ax.set_xlabel("Times")
ax.set_ylabel("AUC")
ax.legend()
ax.axvline(0.0, color="k", linestyle="-")
ax.set_title("Decoding MEG sensors over time")